<font size="5">**Project Summary**</font><br>
In this competition, the main task is to do surface time series classification. 1d convolution is a good technique when doing the time series classification job. So I apply 1d convolutional neural network to solve this problem. The whole code is written in Pytorch.

<font size="5">**External libs we are using**</font><br>

In [85]:
# libs we are using
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

<font size="5">**Data preparations**</font><br>

In this project, I use the raw data as input of the network. I concatenated all datasets into one single numpy array. The first 3810 rows are training data with given labels, the rest 3816 rows are testing data where labels are not given. In total raw data has 7816 rows. In addition to that, we split the raw training data into training data and validation data by the ratio of 80/20.


I create three datasets and data loaders for them to make the data ready for model's training. The process is straightforward. I split the labelled data into two subsets, and keep testing data as is. Also, I convert Numpy arrays into `torch.tensor` objects of proper type (float for samples, and long - for targets).


In [86]:
def create_datasets(data, target, train_size, valid_pct=0.2, seed=None):
    """Converts NumPy arrays into PyTorch datsets.
    
    Three datasets are created in total:
        * training dataset
        * validation dataset
        * testing (un-labelled) dataset

    """
    raw =  data
    sz = train_size
    idx = np.arange(sz)
    trn_idx, val_idx = train_test_split(
        idx, test_size=valid_pct, random_state=seed)
     
    trn_ds = TensorDataset(
        torch.tensor(raw[:sz][trn_idx]).float(), 
        torch.tensor(target[:sz][trn_idx]).long())
    val_ds = TensorDataset(
        torch.tensor(raw[:sz][val_idx]).float(), 
        torch.tensor(target[:sz][val_idx]).long())
    tst_ds = TensorDataset(
        torch.tensor(raw[sz:]).float(), 
        torch.tensor(target[sz:]).long())
    return trn_ds, val_ds, tst_ds

# We use this function to divide the data
def create_datasets2(data_arr, train_size, valid_pct=0.2, seed=None):
    """Converts NumPy arrays into PyTorch datsets.
    
    Three datasets are created in total:
        * training dataset
        * validation dataset
        * testing (un-labelled) dataset

    """
    raw =  data_arr
    sz = train_size

    idx = np.arange(sz)
    print("total data length")
    print(len(idx))
    trn_idx, tst_idx = train_test_split(idx, test_size=0.2, random_state=1)
    trn_idx, val_idx = train_test_split(trn_idx, test_size=0.25, random_state=1)
    
    print("input one dimension shape")
    print(raw[0].shape)
    print("training data length")
    print(len(trn_idx))
    print("validation data le")
    print(len(val_idx))
    print("testing d")
    print(len(tst_idx))
    
    trn_ds = TensorDataset(
        torch.tensor(raw[0][:sz][trn_idx]).float(),
        torch.tensor(raw[1][:sz][trn_idx]).float(),
        torch.tensor(raw[2][:sz][trn_idx]).float(),
        torch.tensor(raw[3][:sz][trn_idx]).float(),
        torch.tensor(raw[4][:sz][trn_idx]).float(),
        torch.tensor(raw[5][:sz][trn_idx]).float(),
        torch.tensor(raw[6][:sz][trn_idx]).float(),
        torch.tensor(raw[7][:sz][trn_idx]).float(),
        torch.tensor(raw[8][:sz][trn_idx]).float(),
        torch.tensor(target[:sz][trn_idx]).long())
    
    val_ds = TensorDataset(
        torch.tensor(raw[0][:sz][val_idx]).float(),
        torch.tensor(raw[1][:sz][val_idx]).float(), 
        torch.tensor(raw[2][:sz][val_idx]).float(), 
        torch.tensor(raw[3][:sz][val_idx]).float(), 
        torch.tensor(raw[4][:sz][val_idx]).float(), 
        torch.tensor(raw[5][:sz][val_idx]).float(), 
        torch.tensor(raw[6][:sz][val_idx]).float(), 
        torch.tensor(raw[7][:sz][val_idx]).float(), 
        torch.tensor(raw[8][:sz][val_idx]).float(), 
        torch.tensor(target[:sz][val_idx]).long())
    
    tst_ds = TensorDataset(
        torch.tensor(raw[0][:sz][tst_idx]).float(),
        torch.tensor(raw[1][:sz][tst_idx]).float(),
        torch.tensor(raw[2][:sz][tst_idx]).float(),
        torch.tensor(raw[3][:sz][tst_idx]).float(),
        torch.tensor(raw[4][:sz][tst_idx]).float(),
        torch.tensor(raw[5][:sz][tst_idx]).float(),
        torch.tensor(raw[6][:sz][tst_idx]).float(),
        torch.tensor(raw[7][:sz][tst_idx]).float(),
        torch.tensor(raw[8][:sz][tst_idx]).float(),
        torch.tensor(target[:sz][tst_idx]).long())
    
    return trn_ds, val_ds, tst_ds

def create_loaders(data, bs=128, jobs=0):
    """Wraps the datasets returned by create_datasets function with data loaders."""
    
    trn_ds, val_ds, tst_ds = data
    trn_dl = DataLoader(trn_ds, batch_size=bs, shuffle=True, num_workers=jobs)
    val_dl = DataLoader(val_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    tst_dl = DataLoader(tst_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    return trn_dl, val_dl, tst_dl

<font size="5">**Neural Network Architecture**</font><br>

In [87]:
class Surface_Classifier(nn.Module):
    def __init__(self, raw_ni, no, drop=.5):
        super().__init__()
        
        self.conv1d_channel_0 = nn.Sequential(
            nn.Conv1d(1, 16, 7, 2, 3),
            nn.BatchNorm1d(16),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    16,  64, 8, 4, 2),
            nn.BatchNorm1d(64),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    64, 128, 8, 4, 2),
            nn.BatchNorm1d(128),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(   128, 256, 8, 4, 2),
            nn.BatchNorm1d(256),
            nn.PReLU(),
            nn.Dropout()
        )
        
        self.conv1d_channel_1 = nn.Sequential(
            nn.Conv1d(1, 16, 7, 2, 3),
            nn.BatchNorm1d(16),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    16,  64, 8, 4, 2),
            nn.BatchNorm1d(64),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    64, 128, 8, 4, 2),
            nn.BatchNorm1d(128),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(   128, 256, 8, 4, 2),
            nn.BatchNorm1d(256),
            nn.PReLU(),
            nn.Dropout()
        )
        
        self.conv1d_channel_2 = nn.Sequential(
            nn.Conv1d(1, 16, 7, 2, 3),
            nn.BatchNorm1d(16),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    16,  64, 8, 4, 2),
            nn.BatchNorm1d(64),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    64, 128, 8, 4, 2),
            nn.BatchNorm1d(128),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(   128, 256, 8, 4, 2),
            nn.BatchNorm1d(256),
            nn.PReLU(),
            nn.Dropout()
        )
        
        self.conv1d_channel_3 = nn.Sequential(
            nn.Conv1d(1, 16, 7, 2, 3),
            nn.BatchNorm1d(16),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    16,  64, 8, 4, 2),
            nn.BatchNorm1d(64),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    64, 128, 8, 4, 2),
            nn.BatchNorm1d(128),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(   128, 256, 8, 4, 2),
            nn.BatchNorm1d(256),
            nn.PReLU(),
            nn.Dropout()
        )
        
        self.conv1d_channel_4 = nn.Sequential(
            nn.Conv1d(1, 16, 7, 2, 3),
            nn.BatchNorm1d(16),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    16,  64, 8, 4, 2),
            nn.BatchNorm1d(64),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    64, 128, 8, 4, 2),
            nn.BatchNorm1d(128),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(   128, 256, 8, 4, 2),
            nn.BatchNorm1d(256),
            nn.PReLU(),
            nn.Dropout()
        )
        
        self.conv1d_channel_5 = nn.Sequential(
            nn.Conv1d(1, 16, 7, 2, 3),
            nn.BatchNorm1d(16),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    16,  64, 8, 4, 2),
            nn.BatchNorm1d(64),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    64, 128, 8, 4, 2),
            nn.BatchNorm1d(128),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(   128, 256, 8, 4, 2),
            nn.BatchNorm1d(256),
            nn.PReLU(),
            nn.Dropout()
        )
        
        self.conv1d_channel_6 = nn.Sequential(
            nn.Conv1d(1, 16, 7, 2, 3),
            nn.BatchNorm1d(16),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    16,  64, 8, 4, 2),
            nn.BatchNorm1d(64),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    64, 128, 8, 4, 2),
            nn.BatchNorm1d(128),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(   128, 256, 8, 4, 2),
            nn.BatchNorm1d(256),
            nn.PReLU(),
            nn.Dropout()
        )
        
        self.conv1d_channel_7 = nn.Sequential(
            nn.Conv1d(1, 16, 7, 2, 3),
            nn.BatchNorm1d(16),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    16,  64, 8, 4, 2),
            nn.BatchNorm1d(64),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    64, 128, 8, 4, 2),
            nn.BatchNorm1d(128),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(   128, 256, 8, 4, 2),
            nn.BatchNorm1d(256),
            nn.PReLU(),
            nn.Dropout()
        )
        
        self.conv1d_channel_8 = nn.Sequential(
            nn.Conv1d(1, 16, 7, 2, 3),
            nn.BatchNorm1d(16),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    16,  64, 8, 4, 2),
            nn.BatchNorm1d(64),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    64, 128, 8, 4, 2),
            nn.BatchNorm1d(128),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(   128, 256, 8, 4, 2),
            nn.BatchNorm1d(256),
            nn.PReLU(),
            nn.Dropout()
        )
        
        self.dense = nn.Sequential(
            nn.Linear(2304, 128), nn.ReLU(), nn.Dropout(),
            nn.Linear(128, 64), nn.Softmax(dim=1), 
            nn.Linear(64, no)
        )
            

    def forward(self, t_channel_0, t_channel_1, t_channel_2, t_channel_3, t_channel_4, t_channel_5, t_channel_6, t_channel_7, t_channel_8):
        conv1d_out_channel_0 = self.conv1d_channel_0(t_channel_0)
        conv1d_out_channel_1 = self.conv1d_channel_1(t_channel_1)
        conv1d_out_channel_2 = self.conv1d_channel_2(t_channel_2)
        conv1d_out_channel_3 = self.conv1d_channel_3(t_channel_3)
        conv1d_out_channel_4 = self.conv1d_channel_4(t_channel_4)
        conv1d_out_channel_5 = self.conv1d_channel_5(t_channel_5)
        conv1d_out_channel_6 = self.conv1d_channel_6(t_channel_6)
        conv1d_out_channel_7 = self.conv1d_channel_7(t_channel_7)
        conv1d_out_channel_8 = self.conv1d_channel_8(t_channel_8)
        
        t_in = torch.cat([conv1d_out_channel_0,conv1d_out_channel_1, conv1d_out_channel_2, conv1d_out_channel_3, conv1d_out_channel_4, conv1d_out_channel_5, conv1d_out_channel_6, conv1d_out_channel_7, conv1d_out_channel_8], dim=1)
        res = t_in.view(t_in.size(0), -1)
       
        out = self.dense(res)
        return out
        

<font size="5">**Initial setup for random seed and project path**</font><br>

In [88]:

# set up the seed
seed = 1
np.random.seed(seed)

# path to sample submission
sample = Path.cwd().parent/'input'/'career-con-2019'/'sample_submission.csv'

ROOT = Path.cwd().parent/'input'/'career-con-2019-preprocessed-data'
raw_arr = np.load(ROOT/'feat.npy').transpose(0, 2, 1)
target = np.load(ROOT/'target.npy')

raw_dim_data = [None]*9
#print(raw_dim_data)
for i in range(0, 9):
    raw_dim_data[i] = raw_arr[:,i,:]
#    print("raw data shape")
    
    raw_dim_data[i] = raw_dim_data[i].reshape([7626,1,128])
#    print(raw_dim_data[i].shape)
    
# print("raw array shape")
# print(raw_arr.shape)
# print("label array shape")
# print(target.shape)

trn_sz = 3810  # only the first `trn_sz` rows in each array include labelled data
#datasets = create_datasets((raw_arr), target, trn_sz, seed=seed)
datasets = create_datasets2((raw_dim_data), trn_sz, seed=seed)


# make sure that we run on a proper device (not relevant for Kaggle kernels but helpful in Jupyter sessions)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


trn_dl, val_dl, tst_dl = create_loaders(datasets, bs=256)

total data length
3810
input one dimension shape
(7626, 1, 128)
training data length
2286
validation data le
762
testing d
762


<font size="5">**Training model**</font><br>
Now everything is ready to create a training loop and see if our model works. For each training epoch, the loop performs the following actions:
1. train model on the `trn_ds` dataset;
2. verify quality on the `val_ds` dataset;
3. check if the quality improved since previous epoch, and if so, save the model's weights onto disk;
4. in case if the model's quality isn't impoving for `patience` epochs, the training is stopped.
Also, the code tracks loss and accuracy history, and prints current scores with exponentially increasing logging frequency, i.e., only at 1, 2, 4, 8... epochs.

In [89]:
raw_feat = raw_arr.shape[1]


lr = 0.001
n_epochs = 10000
iterations_per_epoch = len(trn_dl)
num_classes = 9
best_acc = 0
patience, trials = 500, 0
base = 1
step = 2
loss_history = []
acc_history = []

model = Surface_Classifier(raw_feat, num_classes).to(device)
model.cuda()
criterion = nn.CrossEntropyLoss(reduction='sum')
opt = optim.Adam(model.parameters(), lr=lr)

print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(trn_dl):
        #x_raw, y_batch = [t.to(device) for t in batch]
        x_channel_0, x_channel_1,  x_channel_2,  x_channel_3,  x_channel_4,  x_channel_5,  x_channel_6,  x_channel_7,  x_channel_8, y_batch = [t.to(device) for t in batch]
        opt.zero_grad()
        
#         print("channel 0 type")
#         print(type(x_channel_0))
#         print("channel 0 shape")
#         print(x_channel_0.shape)
#         print("batch type")
#         print(type(batch))
#         print(len(batch))
#         print(batch[0].shape)
#         print(batch[9].shape)

        out = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
        
        
#         out = []
#         with torch.no_grad():
#             for x_channel_0, x_channel_1,  x_channel_2,  x_channel_3,  x_channel_4,  x_channel_5,  x_channel_6,  x_channel_7,  x_channel_8, y_batch in batch:
#                 output = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
#                 out.append(output.detach())
#         out = torch.cat(out)
        

    
        loss = criterion(out, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        opt.step()
        
    epoch_loss /= trn_sz
    loss_history.append(epoch_loss)
    
    model.eval()
    correct, total = 0, 0
    
    for batch in val_dl:
        x_channel_0, x_channel_1,  x_channel_2,  x_channel_3,  x_channel_4,  x_channel_5,  x_channel_6,  x_channel_7,  x_channel_8, y_batch = [t.to(device) for t in batch]
        
        out = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
    acc = correct / total
    acc_history.append(acc)

    if epoch % base == 0:
        print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
        base *= step
    print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')

    
    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break
            
print('Done!')

Start model training
Epoch:   1. Loss: 1.3149. Acc.: 15.09%
Epoch:   1. Loss: 1.3149. Acc.: 15.09%
Epoch 1 best model saved with accuracy: 15.09%
Epoch:   2. Loss: 1.2988. Acc.: 15.09%
Epoch:   2. Loss: 1.2988. Acc.: 15.09%
Epoch:   3. Loss: 1.2833. Acc.: 16.27%
Epoch 3 best model saved with accuracy: 16.27%
Epoch:   4. Loss: 1.2714. Acc.: 17.72%
Epoch:   4. Loss: 1.2714. Acc.: 17.72%
Epoch 4 best model saved with accuracy: 17.72%
Epoch:   5. Loss: 1.2627. Acc.: 20.60%
Epoch 5 best model saved with accuracy: 20.60%
Epoch:   6. Loss: 1.2548. Acc.: 21.65%
Epoch 6 best model saved with accuracy: 21.65%
Epoch:   7. Loss: 1.2480. Acc.: 22.05%
Epoch 7 best model saved with accuracy: 22.05%
Epoch:   8. Loss: 1.2408. Acc.: 22.18%
Epoch:   8. Loss: 1.2408. Acc.: 22.18%
Epoch 8 best model saved with accuracy: 22.18%
Epoch:   9. Loss: 1.2335. Acc.: 22.31%
Epoch 9 best model saved with accuracy: 22.31%
Epoch:  10. Loss: 1.2270. Acc.: 22.18%
Epoch:  11. Loss: 1.2214. Acc.: 22.97%
Epoch 11 best mode

Epoch: 161. Loss: 0.8600. Acc.: 41.08%
Epoch: 162. Loss: 0.8572. Acc.: 41.60%
Epoch: 163. Loss: 0.8549. Acc.: 41.21%
Epoch: 164. Loss: 0.8496. Acc.: 40.94%
Epoch: 165. Loss: 0.8526. Acc.: 40.42%
Epoch: 166. Loss: 0.8541. Acc.: 41.73%
Epoch 166 best model saved with accuracy: 41.73%
Epoch: 167. Loss: 0.8543. Acc.: 42.26%
Epoch 167 best model saved with accuracy: 42.26%
Epoch: 168. Loss: 0.8476. Acc.: 41.60%
Epoch: 169. Loss: 0.8455. Acc.: 41.60%
Epoch: 170. Loss: 0.8457. Acc.: 41.21%
Epoch: 171. Loss: 0.8427. Acc.: 41.47%
Epoch: 172. Loss: 0.8466. Acc.: 41.34%
Epoch: 173. Loss: 0.8451. Acc.: 41.60%
Epoch: 174. Loss: 0.8435. Acc.: 41.86%
Epoch: 175. Loss: 0.8443. Acc.: 42.39%
Epoch 175 best model saved with accuracy: 42.39%
Epoch: 176. Loss: 0.8412. Acc.: 42.26%
Epoch: 177. Loss: 0.8350. Acc.: 41.60%
Epoch: 178. Loss: 0.8312. Acc.: 41.47%
Epoch: 179. Loss: 0.8327. Acc.: 41.99%
Epoch: 180. Loss: 0.8360. Acc.: 42.26%
Epoch: 181. Loss: 0.8271. Acc.: 41.86%
Epoch: 182. Loss: 0.8326. Acc.: 42

Epoch: 354. Loss: 0.6665. Acc.: 44.75%
Epoch: 355. Loss: 0.6611. Acc.: 45.01%
Epoch: 356. Loss: 0.6684. Acc.: 44.49%
Epoch: 357. Loss: 0.6652. Acc.: 44.75%
Epoch: 358. Loss: 0.6529. Acc.: 44.62%
Epoch: 359. Loss: 0.6615. Acc.: 44.75%
Epoch: 360. Loss: 0.6581. Acc.: 45.80%
Epoch: 361. Loss: 0.6620. Acc.: 43.83%
Epoch: 362. Loss: 0.6564. Acc.: 44.36%
Epoch: 363. Loss: 0.6528. Acc.: 44.88%
Epoch: 364. Loss: 0.6448. Acc.: 46.85%
Epoch 364 best model saved with accuracy: 46.85%
Epoch: 365. Loss: 0.6567. Acc.: 45.54%
Epoch: 366. Loss: 0.6540. Acc.: 45.28%
Epoch: 367. Loss: 0.6518. Acc.: 47.38%
Epoch 367 best model saved with accuracy: 47.38%
Epoch: 368. Loss: 0.6475. Acc.: 47.11%
Epoch: 369. Loss: 0.6444. Acc.: 47.64%
Epoch 369 best model saved with accuracy: 47.64%
Epoch: 370. Loss: 0.6513. Acc.: 48.16%
Epoch 370 best model saved with accuracy: 48.16%
Epoch: 371. Loss: 0.6479. Acc.: 49.34%
Epoch 371 best model saved with accuracy: 49.34%
Epoch: 372. Loss: 0.6432. Acc.: 48.43%
Epoch: 373. Lo

Epoch: 528. Loss: 0.4256. Acc.: 64.30%
Epoch 528 best model saved with accuracy: 64.30%
Epoch: 529. Loss: 0.4265. Acc.: 64.57%
Epoch 529 best model saved with accuracy: 64.57%
Epoch: 530. Loss: 0.4235. Acc.: 63.91%
Epoch: 531. Loss: 0.4131. Acc.: 64.04%
Epoch: 532. Loss: 0.4145. Acc.: 64.83%
Epoch 532 best model saved with accuracy: 64.83%
Epoch: 533. Loss: 0.4359. Acc.: 64.44%
Epoch: 534. Loss: 0.4326. Acc.: 64.57%
Epoch: 535. Loss: 0.4080. Acc.: 64.96%
Epoch 535 best model saved with accuracy: 64.96%
Epoch: 536. Loss: 0.4157. Acc.: 63.78%
Epoch: 537. Loss: 0.4141. Acc.: 64.44%
Epoch: 538. Loss: 0.4128. Acc.: 63.65%
Epoch: 539. Loss: 0.4223. Acc.: 63.91%
Epoch: 540. Loss: 0.4120. Acc.: 65.09%
Epoch 540 best model saved with accuracy: 65.09%
Epoch: 541. Loss: 0.4223. Acc.: 64.83%
Epoch: 542. Loss: 0.4059. Acc.: 64.57%
Epoch: 543. Loss: 0.4011. Acc.: 63.78%
Epoch: 544. Loss: 0.4064. Acc.: 64.04%
Epoch: 545. Loss: 0.3932. Acc.: 64.17%
Epoch: 546. Loss: 0.4054. Acc.: 64.83%
Epoch: 547. Lo

Epoch: 723. Loss: 0.2543. Acc.: 66.93%
Epoch: 724. Loss: 0.2604. Acc.: 66.67%
Epoch: 725. Loss: 0.2479. Acc.: 65.35%
Epoch: 726. Loss: 0.2632. Acc.: 65.49%
Epoch: 727. Loss: 0.2467. Acc.: 66.01%
Epoch: 728. Loss: 0.2453. Acc.: 66.27%
Epoch: 729. Loss: 0.2652. Acc.: 65.09%
Epoch: 730. Loss: 0.2476. Acc.: 65.09%
Epoch: 731. Loss: 0.2478. Acc.: 64.96%
Epoch: 732. Loss: 0.2511. Acc.: 66.40%
Epoch: 733. Loss: 0.2569. Acc.: 67.72%
Epoch 733 best model saved with accuracy: 67.72%
Epoch: 734. Loss: 0.2543. Acc.: 67.59%
Epoch: 735. Loss: 0.2484. Acc.: 67.72%
Epoch: 736. Loss: 0.2632. Acc.: 67.32%
Epoch: 737. Loss: 0.2554. Acc.: 66.01%
Epoch: 738. Loss: 0.2478. Acc.: 66.14%
Epoch: 739. Loss: 0.2540. Acc.: 66.67%
Epoch: 740. Loss: 0.2408. Acc.: 65.88%
Epoch: 741. Loss: 0.2534. Acc.: 66.14%
Epoch: 742. Loss: 0.2476. Acc.: 66.27%
Epoch: 743. Loss: 0.2525. Acc.: 65.09%
Epoch: 744. Loss: 0.2573. Acc.: 66.14%
Epoch: 745. Loss: 0.2578. Acc.: 66.40%
Epoch: 746. Loss: 0.2527. Acc.: 65.88%
Epoch: 747. Los

...

Epoch: 1735. Loss: 0.0905. Acc.: 72.44%
Epoch: 1736. Loss: 0.0894. Acc.: 72.18%
Epoch: 1737. Loss: 0.0900. Acc.: 72.44%
Epoch: 1738. Loss: 0.0855. Acc.: 73.10%
Epoch: 1739. Loss: 0.0858. Acc.: 74.15%
Epoch: 1740. Loss: 0.0822. Acc.: 74.28%
Epoch: 1741. Loss: 0.0749. Acc.: 74.02%
Epoch: 1742. Loss: 0.0807. Acc.: 73.49%
Epoch: 1743. Loss: 0.0724. Acc.: 73.62%
Epoch: 1744. Loss: 0.0949. Acc.: 73.88%
Epoch: 1745. Loss: 0.0870. Acc.: 73.62%
Epoch: 1746. Loss: 0.0928. Acc.: 74.02%
Epoch: 1747. Loss: 0.0982. Acc.: 73.75%
Epoch: 1748. Loss: 0.0778. Acc.: 72.97%
Epoch: 1749. Loss: 0.0859. Acc.: 73.36%
Epoch: 1750. Loss: 0.0911. Acc.: 72.70%
Epoch: 1751. Loss: 0.0906. Acc.: 72.97%
Epoch: 1752. Loss: 0.0921. Acc.: 72.83%
Epoch: 1753. Loss: 0.0863. Acc.: 73.23%
Epoch: 1754. Loss: 0.0939. Acc.: 74.15%
Epoch: 1755. Loss: 0.0925. Acc.: 73.49%
Epoch: 1756. Loss: 0.0893. Acc.: 73.49%
Epoch: 1757. Loss: 0.0848. Acc.: 73.49%
Epoch: 1758. Loss: 0.0710. Acc.: 73.49%
Epoch: 1759. Loss: 0.0784. Acc.: 73.49%


Epoch: 1940. Loss: 0.0870. Acc.: 72.70%
Epoch: 1941. Loss: 0.0824. Acc.: 73.23%
Epoch: 1942. Loss: 0.0866. Acc.: 73.23%
Epoch: 1943. Loss: 0.0827. Acc.: 72.57%
Epoch: 1944. Loss: 0.0747. Acc.: 72.97%
Epoch: 1945. Loss: 0.0775. Acc.: 72.44%
Epoch: 1946. Loss: 0.0785. Acc.: 72.57%
Epoch: 1947. Loss: 0.0827. Acc.: 72.57%
Epoch: 1948. Loss: 0.0808. Acc.: 72.97%
Epoch: 1949. Loss: 0.0839. Acc.: 72.44%
Epoch: 1950. Loss: 0.0810. Acc.: 72.31%
Epoch: 1951. Loss: 0.0884. Acc.: 71.92%
Epoch: 1952. Loss: 0.0895. Acc.: 72.70%
Epoch: 1953. Loss: 0.0921. Acc.: 72.57%
Epoch: 1954. Loss: 0.0808. Acc.: 72.57%
Epoch: 1955. Loss: 0.0859. Acc.: 72.97%
Epoch: 1956. Loss: 0.0775. Acc.: 71.78%
Epoch: 1957. Loss: 0.0824. Acc.: 72.70%
Epoch: 1958. Loss: 0.0760. Acc.: 72.83%
Epoch: 1959. Loss: 0.0807. Acc.: 73.88%
Epoch: 1960. Loss: 0.0815. Acc.: 73.75%
Epoch: 1961. Loss: 0.0795. Acc.: 72.97%
Epoch: 1962. Loss: 0.0783. Acc.: 72.97%
Epoch: 1963. Loss: 0.0818. Acc.: 73.10%
Epoch: 1964. Loss: 0.0804. Acc.: 72.97%


Epoch: 2143. Loss: 0.0742. Acc.: 73.75%
Epoch: 2144. Loss: 0.0798. Acc.: 72.57%
Epoch: 2145. Loss: 0.0849. Acc.: 73.10%
Epoch: 2146. Loss: 0.0696. Acc.: 73.49%
Epoch: 2147. Loss: 0.0763. Acc.: 73.10%
Epoch: 2148. Loss: 0.0726. Acc.: 74.15%
Epoch: 2149. Loss: 0.0765. Acc.: 74.67%
Epoch: 2150. Loss: 0.0662. Acc.: 74.28%
Epoch: 2151. Loss: 0.0687. Acc.: 74.15%
Epoch: 2152. Loss: 0.0687. Acc.: 73.36%
Epoch: 2153. Loss: 0.0706. Acc.: 73.88%
Epoch: 2154. Loss: 0.0936. Acc.: 73.36%
Epoch: 2155. Loss: 0.0773. Acc.: 72.18%
Epoch: 2156. Loss: 0.0733. Acc.: 72.18%
Epoch: 2157. Loss: 0.0795. Acc.: 72.97%
Epoch: 2158. Loss: 0.0765. Acc.: 73.10%
Epoch: 2159. Loss: 0.0655. Acc.: 72.31%
Epoch: 2160. Loss: 0.0766. Acc.: 73.23%
Epoch: 2161. Loss: 0.0689. Acc.: 72.70%
Epoch: 2162. Loss: 0.0660. Acc.: 72.44%
Epoch: 2163. Loss: 0.0669. Acc.: 72.05%
Epoch: 2164. Loss: 0.0728. Acc.: 72.83%
Epoch: 2165. Loss: 0.0663. Acc.: 72.57%
Epoch: 2166. Loss: 0.0662. Acc.: 73.10%
Epoch: 2167. Loss: 0.0748. Acc.: 73.62%


Epoch: 2348. Loss: 0.0635. Acc.: 73.49%
Epoch: 2349. Loss: 0.0748. Acc.: 73.75%
Epoch: 2350. Loss: 0.0692. Acc.: 74.02%
Epoch: 2351. Loss: 0.0691. Acc.: 72.05%
Epoch: 2352. Loss: 0.0711. Acc.: 71.26%
Epoch: 2353. Loss: 0.0693. Acc.: 71.52%
Epoch: 2354. Loss: 0.0753. Acc.: 71.78%
Epoch: 2355. Loss: 0.0741. Acc.: 72.05%
Epoch: 2356. Loss: 0.0616. Acc.: 72.18%
Epoch: 2357. Loss: 0.0692. Acc.: 72.18%
Epoch: 2358. Loss: 0.0625. Acc.: 72.31%
Epoch: 2359. Loss: 0.0648. Acc.: 72.18%
Epoch: 2360. Loss: 0.0643. Acc.: 72.97%
Epoch: 2361. Loss: 0.0713. Acc.: 72.44%
Epoch: 2362. Loss: 0.0700. Acc.: 72.31%
Epoch: 2363. Loss: 0.0702. Acc.: 72.05%
Epoch: 2364. Loss: 0.0636. Acc.: 71.52%
Epoch: 2365. Loss: 0.0549. Acc.: 72.05%
Epoch: 2366. Loss: 0.0677. Acc.: 72.31%
Epoch: 2367. Loss: 0.0646. Acc.: 70.87%
Epoch: 2368. Loss: 0.0631. Acc.: 71.13%
Epoch: 2369. Loss: 0.0657. Acc.: 72.57%
Epoch: 2370. Loss: 0.0634. Acc.: 71.92%
Epoch: 2371. Loss: 0.0678. Acc.: 71.13%
Epoch: 2372. Loss: 0.0669. Acc.: 71.65%


Epoch: 2553. Loss: 0.0544. Acc.: 74.54%
Epoch: 2554. Loss: 0.0607. Acc.: 73.10%
Epoch: 2555. Loss: 0.0577. Acc.: 72.57%
Epoch: 2556. Loss: 0.0532. Acc.: 73.23%
Epoch: 2557. Loss: 0.0540. Acc.: 73.62%
Epoch: 2558. Loss: 0.0571. Acc.: 73.62%
Epoch: 2559. Loss: 0.0592. Acc.: 73.23%
Epoch: 2560. Loss: 0.0607. Acc.: 72.97%
Epoch: 2561. Loss: 0.0457. Acc.: 74.80%
Epoch: 2562. Loss: 0.0518. Acc.: 73.62%
Epoch: 2563. Loss: 0.0617. Acc.: 74.15%
Epoch: 2564. Loss: 0.0594. Acc.: 74.02%
Epoch: 2565. Loss: 0.0583. Acc.: 74.15%
Epoch: 2566. Loss: 0.0580. Acc.: 74.02%
Epoch: 2567. Loss: 0.0560. Acc.: 73.23%
Epoch: 2568. Loss: 0.0516. Acc.: 72.31%
Epoch: 2569. Loss: 0.0552. Acc.: 73.36%
Epoch: 2570. Loss: 0.0574. Acc.: 73.62%
Epoch: 2571. Loss: 0.0623. Acc.: 73.75%
Epoch: 2572. Loss: 0.0615. Acc.: 73.75%
Epoch: 2573. Loss: 0.0506. Acc.: 74.02%
Epoch: 2574. Loss: 0.0558. Acc.: 72.44%
Epoch: 2575. Loss: 0.0635. Acc.: 73.49%
Epoch: 2576. Loss: 0.0559. Acc.: 73.88%
Epoch: 2577. Loss: 0.0635. Acc.: 74.67%


<font size="5">**Compute result on testing data**</font><br>

In [90]:
test_results = []
model.load_state_dict(torch.load('best.pth'))
model.eval()

# for batch in val_dl:
#     x_raw, y_batch = [t.to(device) for t in batch]
#     out = model(x_raw)
#     preds = F.log_softmax(out, dim=1).argmax(dim=1)
#     total += y_batch.size(0)
#     correct += (preds == y_batch).sum().item()
#     acc = correct / total
#     acc_history.append(acc)

# for batch in tst_dl:
#     x, y_batche = [t.to(device) for t in (batch)]
#     out = model(x)
#     y_hat = F.log_softmax(out, dim=1).argmax(dim=1)
#     test_results.extend(y_hat.tolist())
    
for batch in tst_dl:
        x_raw, y_batch = [t.to(device) for t in batch]
        out = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
acc = correct / total
print("accurancy on test data: "+str(acc))

ValueError: too many values to unpack (expected 2)

<font size="5">**Store final result**</font><br>

In [ ]:
'''
submit = pd.read_csv(sample)
enc = joblib.load(ROOT/'encoder.model')
submit['surface'] = enc.inverse_transform(test_results)
submit.to_csv('submit_base.csv', index=None)
print("store result successfully!")
'''